# APMA2822B Homework 1 - Hammad Izhar + Robert Scheidegger

In this report we plan to analyze the performance of matrix-vector multiplication using a variety of memory allocation patterns and multiplication methods. These experiments were conducted using Brown's compute grid OSCAR using 32 cores [insert cpu name] and 4GB of RAM [insert ram specification].

A total of 2016 configurations were run varying memory allocators, multiplication methods, size of matrix, and number of threads. A summary of memory allocators and multiplication methods used is given below:

<center>

| Memory Allocator              | Description |
| ----------------------------- | ----------- |
| `DisjointMemoryAllocator`     |             |
| `DisjointRowMemoryAllocator`  |             |
| `ContiguousMemoryAllocator`   |             |
| `MmapMemoryAllocator`         |             |

<br/>

| Multipliers            | Description |
| ---------------------- | ----------- |
| `RowColumnMultiplier`  |             |
| `ColumnRowMultiplier`  |             |
</center>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
df = pd.read_csv('../data/oscar_data.csv', encoding='latin-1')
df['flops'] = 2 * df['n'] * df['n'] / df['time_us'] * 1e6
df['gflops'] = df['flops'] / 1e9
df['iops'] = 5 * df['n'] * df['m'] / df['time_us'] * 1e6

## Roofline Analysis

For the purposes of this analysis, we will look at the results from $100000$-by-$10000$ matrices allocated using the `DisjointMemoryAllocator` and multiplied using the `RowColumnMultiplier`. These are the largest matrices we allocated for these experiments.

Excluding timing and parallelization primitives, the code of `RowColumnMultiplier` is as follows:

```c++
// include/multipliers.hpp

for (uint32_t i = 0; i < n; i++) {
    for (uint32_t j = 0; j < m; j++) {
        output[i] += matrix[i][j] * vector[j];
    }
}
```

To compute the arithmetic intensity, we first count the number of I/O operations (memory accesses) required:

1. Load `matrix[i]` (since this is a pointer, and is needed to find the column value)
2. Load `matrix[i][j]`
3. Load `vector[j]`
4. Save `matrix[i][j] * vector[j]` into a temporary variable
5. Load `output[i]` (can add this to the temporary variable)
6. Save `output[i]`

This gives 6 I/O operations, each acting on 4 bytes (since these are all `float` values), giving a total of `24` bytes transferred. Further, the floating point operations are:

1. Multiply `RESULT = matrix[i][j] * vector[j]`
2. Add `RESULT + output[i]`

This gives an overall arithmetic intensity (AI) of `AI = 2 / 24 = 1 / 12`. 

Assuming that the CPU has a peak memory bandwidth of 160 GB/s and a peak FLOP rate of 1 TFLOP/s, we can compute the limit point for our roofline plot.
$$
    x^* = \frac{10^{12}}{160 \cdot 10^9} = 6.25
$$
This indicates that we are in the bandwidth-limited region of the roof-line plot. Our expected peak FLOP rate is therefore
$$
    \text{Peak GFLOP Rate} = \left(1000 \ \frac{\text{GFLOP}}{\text{second}}\right) \left(\frac{1}{12}\right) = 83.3 \ \text{GFLOP}
$$

By the example numbers given we should expect the limiting point of the roof-line plot to be at `10^12 / (160 * 10^9) = 6.25`, so clearly we are within the memory bandwidth limiting section of the roof-line plot. Thus, we should expect a peak GFLOPs rate of `1000 * (1/12) = 83.3` GFLOPs. 

This is corroborated experimentally when testing out matrix-vector multiplication using 32 threads. The peak GFLOP rate is 110 GFLOP. The difference between the results can be attributed to the fact that the CPU specification estimates aren't reflective of the true OSCAR hardware specification.

This is all summarized in the plot below.

In [9]:
# Compute the roof-line plot for the analysis above.
subset = df[df['allocator'] == 'DisjointMemoryAllocator']
subset = subset[subset['multiplier'] == 'RowColumnMultiplier']
subset = subset[subset['m'] == 10000]
subset = subset[subset['n'] == 100000]

# Plot the roof-line plot
# TODO: Hammad

subset



,n,m,threads,allocator,multiplier,iterations,time_us,stdev_us,flops,gflops,iops
1904,100000,10000,1,DisjointMemoryAllocator,RowColumnMultiplier,10,2.730146e+06,0.000000,7.325617e+09,7.325617,1.831404e+09
1912,100000,10000,2,DisjointMemoryAllocator,RowColumnMultiplier,10,1.369082e+06,2231.756348,1.460832e+10,14.608325,3.652081e+09
1920,100000,10000,4,DisjointMemoryAllocator,RowColumnMultiplier,10,6.844341e+05,0.000000,2.922122e+10,29.221220,7.305305e+09
1928,100000,10000,8,DisjointMemoryAllocator,RowColumnMultiplier,10,3.424635e+05,90.509666,5.840038e+10,58.400384,1.460010e+10
1936,100000,10000,16,DisjointMemoryAllocator,RowColumnMultiplier,10,2.091755e+05,45619.386719,9.561349e+10,95.613492,2.390337e+10
1944,100000,10000,32,DisjointMemoryAllocator,RowColumnMultiplier,10,1.816549e+05,11106.904297,1.100989e+11,110.098871,2.752472e+10
1952,100000,10000,64,DisjointMemoryAllocator,RowColumnMultiplier,10,2.033748e+05,5928.901367,9.834060e+10,98.340602,2.458515e+10
1960,100000,10000,1,DisjointMemoryAllocator,RowColumnMultiplier,10,2.730905e+06,0.000000,7.323580e+09,7.323580,1.830895e+09
1968,100000,10000,2,DisjointMemoryAllocator,RowColumnMultiplier,10,1.368980e+06,627.069397,1.460941e+10,14.609412,3.652353e+09
1976,100000,10000,4,DisjointMemoryAllocator,RowColumnMultiplier,10,6.843876e+05,0.000000,2.922321e+10,29.223205,7.305801e+09


## Performance Analysis

To test our

This leads to a total of 2016 possible combinations of parameters, and for each of these we performed an experiment on oscar (in a single script, which iterated through all of the possible benchmark configurations). A warmup/dummy computation was added to each  was added 

To ensure consistency within a run, each was repeated for `10` iterations, and the mean and standard deviations of the runtimes were computed. A sample of the data that we collected is seen below:

In [31]:
df[df['time_us'] != 0].sort_values('flops', ascending=False)

,n,m,threads,allocator,multiplier,iterations,time_us,stdev_us,flops,gflops,iops
1708,100000,1,8,ContiguousMemoryAllocator,RowColumnMultiplier,10,50.099998,2.022457,3.992016e+14,3.992016e+05,9.980040e+09
1706,100000,1,8,DisjointRowMemoryAllocator,RowColumnMultiplier,10,85.500000,10.452272,2.339181e+14,2.339181e+05,5.847953e+09
1700,100000,1,4,ContiguousMemoryAllocator,RowColumnMultiplier,10,93.800003,2.357874,2.132196e+14,2.132196e+05,5.330490e+09
1704,100000,1,8,DisjointMemoryAllocator,RowColumnMultiplier,10,110.000000,35.437267,1.818182e+14,1.818182e+05,4.545455e+09
1724,100000,1,32,ContiguousMemoryAllocator,RowColumnMultiplier,10,126.199997,26.049194,1.584786e+14,1.584786e+05,3.961965e+09
...,...,...,...,...,...,...,...,...,...,...,...
91,1,10,16,DisjointRowMemoryAllocator,ColumnRowMultiplier,10,8447.700195,1202.128174,2.367508e+02,2.367508e-07,5.918771e+03
313,1,10000,16,DisjointMemoryAllocator,ColumnRowMultiplier,10,8459.099609,1211.121826,2.364318e+02,2.364318e-07,5.910795e+06
262,1,10000,16,MmapMemoryAllocator,RowColumnMultiplier,10,8461.599609,1215.756592,2.363619e+02,2.363619e-07,5.909048e+06
256,1,10000,16,DisjointMemoryAllocator,RowColumnMultiplier,10,8489.299805,1232.574585,2.355907e+02,2.355907e-07,5.889767e+06
